### Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cvxpy as cp
from tqdm import tqdm
import timesfm

%load_ext autoreload
%autoreload 2
%matplotlib inline

 See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded PyTorch TimesFM, likely because python version is 3.11.5 (main, Sep 11 2023, 13:54:46) [GCC 11.2.0].


In [2]:
look_ahead = 12*10
look_back = 12*24*7

In [ ]:
# Loading the timesfm-2.0 checkpoint:
# For Torch
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="gpu",
          per_core_batch_size=32,
          horizon_len=look_ahead,
          num_layers=50,
          use_positional_embedding=False,
          context_len=look_back,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
cwd = os.getcwd()

def make_dir(path):
    if os.path.exists(path) is False:
        os.makedirs(path)

def evaluate_prediction(predictions, actual, model_name):
    errors = predictions - actual
    mse = np.square(errors).mean()
    rmse = np.sqrt(mse)
    mae = np.abs(errors).mean()
    print('MAE: {:.2f}'.format(mae))
    print('RMSE: {:.2f}'.format(rmse))
    print('')
    print('')
    return mae, rmse

### Preprocessing
Prepare the features and the label. We have two timeseries: sced (real-time) system lambda and day-ahead (DAP) system lambda. We want to predict the real-time system lambda for certain horizon (e.g. 6 hours ahead = 72 points in 5-mins). DAP are available more than 24 hours ahead.  

In [4]:
# Define the slice length (e.g., 4 years + 1 day)
slice_length = 12 * 24 * 365 * 4 + 12 * 24
slice_length_diff = 12 * 24 * 365 * 4 #+ 12 * 24 #lost on day in 2019 for the diff

In [5]:
# prepare the nodal real-time price in the required shape 
actual = pd.read_csv('lmp_WAKEWE_ALL_2023.csv')
actual_rolled = pd.DataFrame([pd.Series(actual['price'].iloc[i+1:i+1+look_ahead].values) for i in range(len(actual) - look_ahead)])
actual_rolled.columns = [f'{i+1}' for i in range(look_ahead)]
actual_rolled['timestamp'] = actual['timestamp'].iloc[:len(actual_rolled)].values
actual_rolled = actual_rolled[['timestamp'] + [col for col in actual_rolled.columns if col != 'timestamp']]
actual_rolled = actual_rolled.iloc[:,:look_ahead+1]

In [6]:
actual_rolled

,timestamp,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,2023-01-01 00:00:00,-2.65,-2.39,-2.35,-2.34,-2.33,-2.33,-2.02,-1.54,-1.69,...,2.86,2.74,2.78,9.25,9.13,9.12,9.26,11.77,19.33,20.83
1,2023-01-01 00:05:00,-2.39,-2.35,-2.34,-2.33,-2.33,-2.02,-1.54,-1.69,-1.69,...,2.74,2.78,9.25,9.13,9.12,9.26,11.77,19.33,20.83,20.21
2,2023-01-01 00:10:00,-2.35,-2.34,-2.33,-2.33,-2.02,-1.54,-1.69,-1.69,-1.53,...,2.78,9.25,9.13,9.12,9.26,11.77,19.33,20.83,20.21,19.53
3,2023-01-01 00:15:00,-2.34,-2.33,-2.33,-2.02,-1.54,-1.69,-1.69,-1.53,-1.53,...,9.25,9.13,9.12,9.26,11.77,19.33,20.83,20.21,19.53,23.22
4,2023-01-01 00:20:00,-2.33,-2.33,-2.02,-1.54,-1.69,-1.69,-1.53,-1.53,-1.53,...,9.13,9.12,9.26,11.77,19.33,20.83,20.21,19.53,23.22,24.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,2023-12-31 13:35:00,12.29,11.97,11.28,11.78,11.75,11.19,8.19,8.97,10.76,...,14.07,13.84,14.19,14.40,14.44,14.87,14.76,14.48,14.73,14.82
104996,2023-12-31 13:40:00,11.97,11.28,11.78,11.75,11.19,8.19,8.97,10.76,11.24,...,13.84,14.19,14.40,14.44,14.87,14.76,14.48,14.73,14.82,14.54
104997,2023-12-31 13:45:00,11.28,11.78,11.75,11.19,8.19,8.97,10.76,11.24,10.47,...,14.19,14.40,14.44,14.87,14.76,14.48,14.73,14.82,14.54,14.47
104998,2023-12-31 13:50:00,11.78,11.75,11.19,8.19,8.97,10.76,11.24,10.47,7.73,...,14.40,14.44,14.87,14.76,14.48,14.73,14.82,14.54,14.47,14.39


In [7]:
# read and clean real-time system lambda
sl = pd.read_csv('ercot_sl_2019_2023.csv')
sl['sced_time_stamp_local'] = pd.to_datetime(sl['sced_time_stamp_local'])
sl.set_index('sced_time_stamp_local', inplace=True)
sl = sl.resample('5min').interpolate()
date_range = pd.date_range(start=sl.index.min(), end=sl.index.max(), freq='5min')
sl = sl[~sl.index.duplicated(keep='first')]
sl = sl.reindex(date_range, fill_value=np.nan)
sl.interpolate(method='time', inplace=True)

In [8]:
# prepare the nodal real-time price in the required shape 
actual_sl = pd.read_csv('ercot_sl_2019_2023.csv').iloc[slice_length:]
actual_sl_rolled = pd.DataFrame([pd.Series(actual_sl['system_lambda'].iloc[i+1:i+1+look_ahead].values) for i in range(len(actual_sl) - look_ahead)])
actual_sl_rolled.columns = [f'{i+1}' for i in range(look_ahead)]
actual_sl_rolled['sced_time_stamp_local'] = actual_sl['sced_time_stamp_local'].iloc[:len(actual_sl_rolled)].values
actual_sl_rolled = actual_sl_rolled[['sced_time_stamp_local'] + [col for col in actual_sl_rolled.columns if col != 'sced_time_stamp_local']]
actual_sl_rolled = actual_sl_rolled.iloc[:,:look_ahead+1]

In [9]:
actual_sl_rolled

,sced_time_stamp_local,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,2023-01-01 00:00:00,-2.648146,-2.390966,-2.349594,-2.343839,-2.326401,-2.321906,-1.985557,-1.454155,-1.615148,...,2.858775,2.740664,2.780005,13.032491,13.377475,13.262575,12.408667,17.013020,24.164930,20.830612
1,2023-01-01 00:05:00,-2.390966,-2.349594,-2.343839,-2.326401,-2.321906,-1.985557,-1.454155,-1.615148,-1.612673,...,2.740664,2.780005,13.032491,13.377475,13.262575,12.408667,17.013020,24.164930,20.830612,20.210316
2,2023-01-01 00:10:00,-2.349594,-2.343839,-2.326401,-2.321906,-1.985557,-1.454155,-1.615148,-1.612673,-1.451050,...,2.780005,13.032491,13.377475,13.262575,12.408667,17.013020,24.164930,20.830612,20.210316,19.526390
3,2023-01-01 00:15:00,-2.343839,-2.326401,-2.321906,-1.985557,-1.454155,-1.615148,-1.612673,-1.451050,-1.450721,...,13.032491,13.377475,13.262575,12.408667,17.013020,24.164930,20.830612,20.210316,19.526390,23.218176
4,2023-01-01 00:20:00,-2.326401,-2.321906,-1.985557,-1.454155,-1.615148,-1.612673,-1.451050,-1.450721,-1.453138,...,13.377475,13.262575,12.408667,17.013020,24.164930,20.830612,20.210316,19.526390,23.218176,24.094515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,2023-12-31 13:35:00,11.599060,11.692280,10.586460,11.005346,11.041732,10.374124,7.520594,8.271972,10.110160,...,12.066859,11.847058,12.209281,12.418777,12.466784,12.896057,12.776202,12.563949,12.807795,12.889796
104996,2023-12-31 13:40:00,11.692280,10.586460,11.005346,11.041732,10.374124,7.520594,8.271972,10.110160,10.947229,...,11.847058,12.209281,12.418777,12.466784,12.896057,12.776202,12.563949,12.807795,12.889796,12.632624
104997,2023-12-31 13:45:00,10.586460,11.005346,11.041732,10.374124,7.520594,8.271972,10.110160,10.947229,10.220093,...,12.209281,12.418777,12.466784,12.896057,12.776202,12.563949,12.807795,12.889796,12.632624,12.572374
104998,2023-12-31 13:50:00,11.005346,11.041732,10.374124,7.520594,8.271972,10.110160,10.947229,10.220093,7.666133,...,12.418777,12.466784,12.896057,12.776202,12.563949,12.807795,12.889796,12.632624,12.572374,12.493818


In [10]:
dap = pd.read_csv('hourly_ercot_day_ahead_sl_2019_2023.csv')
dap['timestamp'] = pd.to_datetime(dap['timestamp'])
dap.set_index('timestamp', inplace=True)
dap = dap.resample('5min').ffill()
date_range = pd.date_range(start=dap.index.min(), end='2024-01-01 23:55:00', freq='5min')
dap = dap[~dap.index.duplicated(keep='first')]
dap = dap.reindex(date_range, fill_value=np.nan)
dap.interpolate(method='time', inplace=True)

In [11]:
dap

,SystemLambda
2019-01-02 00:00:00,23.9250
2019-01-02 00:05:00,23.9250
2019-01-02 00:10:00,23.9250
2019-01-02 00:15:00,23.9250
2019-01-02 00:20:00,23.9250
...,...
2024-01-01 23:35:00,19.2456
2024-01-01 23:40:00,19.2456
2024-01-01 23:45:00,19.2456
2024-01-01 23:50:00,19.2456


In [12]:
# prepare the nodal real-time price in the required shape 
actual_dap = dap.iloc[slice_length_diff:-288]
actual_dap_rolled = pd.DataFrame([pd.Series(actual_dap['SystemLambda'].iloc[i+1:i+1+look_ahead].values) for i in range(len(actual_dap) - look_ahead)])
actual_dap_rolled.columns = [f'{i+1}' for i in range(look_ahead)]
actual_dap_rolled['sced_time_stamp_local'] = actual_dap.index[:len(actual_dap_rolled)].values
actual_dap_rolled = actual_dap_rolled[['sced_time_stamp_local'] + [col for col in actual_dap_rolled.columns if col != 'sced_time_stamp_local']]
actual_dap_rolled = actual_dap_rolled.iloc[:,:look_ahead+1]

In [13]:
actual_dap_rolled

,sced_time_stamp_local,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,2023-01-01 00:00:00,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,...,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,17.2491
1,2023-01-01 00:05:00,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,...,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,17.2491,17.2491
2,2023-01-01 00:10:00,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,...,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,17.2491,17.2491,17.2491
3,2023-01-01 00:15:00,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.2222,...,15.0522,15.0522,15.0522,15.0522,15.0522,15.0522,17.2491,17.2491,17.2491,17.2491
4,2023-01-01 00:20:00,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.8334,10.2222,10.2222,...,15.0522,15.0522,15.0522,15.0522,15.0522,17.2491,17.2491,17.2491,17.2491,17.2491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,2023-12-31 13:35:00,12.5549,12.5549,12.5549,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,...,15.7699,15.7699,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404
104996,2023-12-31 13:40:00,12.5549,12.5549,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,...,15.7699,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404
104997,2023-12-31 13:45:00,12.5549,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,...,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404
104998,2023-12-31 13:50:00,12.5549,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,12.1478,...,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404,16.2404


In [6]:
df = pd.concat([dap, sl], axis=1)
df.columns = ['DAP', 'SCED']
df['DAP'] = df['DAP']
df.dropna(inplace=True)

In [7]:
df['diff'] = df['SCED'] - df['DAP']

In [8]:
dap

,SystemLambda
2019-01-02 00:00:00,23.9250
2019-01-02 00:05:00,23.9250
2019-01-02 00:10:00,23.9250
2019-01-02 00:15:00,23.9250
2019-01-02 00:20:00,23.9250
...,...
2024-01-01 23:35:00,19.2456
2024-01-01 23:40:00,19.2456
2024-01-01 23:45:00,19.2456
2024-01-01 23:50:00,19.2456


In [9]:
df

,DAP,SCED,diff
2019-01-02 00:00:00,23.9250,26.159285,2.234285
2019-01-02 00:05:00,23.9250,26.121857,2.196857
2019-01-02 00:10:00,23.9250,26.201380,2.276380
2019-01-02 00:15:00,23.9250,26.343931,2.418931
2019-01-02 00:20:00,23.9250,26.338829,2.413829
...,...,...,...
2023-12-31 23:35:00,16.2404,12.889796,-3.350604
2023-12-31 23:40:00,16.2404,12.632624,-3.607776
2023-12-31 23:45:00,16.2404,12.572374,-3.668026
2023-12-31 23:50:00,16.2404,12.493818,-3.746582


In [ ]:

# Create a list of sequential slices
sequential_slices = [
    df.iloc[slice_length_diff - 12 * 24 * 7 + i:slice_length_diff + i,-1].values
    for i in range(len(df) - slice_length_diff - 120)
]

In [29]:
point_forecasts = tfm.forecast(sequential_slices, freq=[0] * len(sequential_slices))


In [ ]:
pd.DataFrame(point_forecasts[0]).to_csv('TimesFM_120_diff_2023.csv')

In [121]:
pred_diff_df = pd.DataFrame(point_forecasts[0])

In [122]:
pred_diff_df.columns = [f'{i+1}' for i in range(look_ahead)]


In [123]:
pred_diff_df['sced_time_stamp_local'] = actual_dap_rolled['sced_time_stamp_local']

In [124]:
pred_diff_df = pred_diff_df[['sced_time_stamp_local'] + [col for col in pred_diff_df.columns if col != 'sced_time_stamp_local']]

In [125]:
pred_diff_df

,sced_time_stamp_local,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,2023-01-01 00:00:00,-9.874214,-9.899047,-10.109543,-10.024799,-9.958197,-9.795633,-9.980198,-10.083294,-9.994350,...,-8.768623,-8.608265,-8.619745,-8.557821,-8.635050,-8.533489,-8.265808,-8.011129,-7.995575,-7.880092
1,2023-01-01 00:05:00,-13.377350,-13.394665,-13.480650,-13.295856,-13.166882,-12.971603,-13.072046,-13.061566,-12.859574,...,-9.840890,-9.614666,-9.578562,-9.474209,-9.462227,-9.335657,-8.954632,-8.768228,-8.815351,-8.713397
2,2023-01-01 00:10:00,-13.766260,-13.863090,-13.928466,-13.773693,-13.681342,-13.479528,-13.549039,-13.518769,-13.300652,...,-9.974727,-9.739826,-9.652720,-9.468077,-9.408363,-9.297503,-9.033337,-8.966934,-9.090040,-9.074520
3,2023-01-01 00:15:00,-13.676354,-13.786121,-13.841637,-13.699418,-13.654809,-13.439289,-13.456970,-13.408062,-13.124491,...,-10.020098,-9.773392,-9.673538,-9.451614,-9.454635,-9.439824,-9.300214,-9.276365,-9.404181,-9.435984
4,2023-01-01 00:20:00,-13.607414,-13.696375,-13.706566,-13.592072,-13.561863,-13.354302,-13.354043,-13.280640,-13.014064,...,-10.070434,-9.841808,-9.746205,-9.584043,-9.705168,-9.741905,-9.689569,-9.713490,-9.829688,-9.869946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,2023-12-31 13:35:00,-1.332778,-1.401761,-1.550917,-1.590137,-1.741685,-1.940698,-2.025429,-2.095047,-2.261121,...,-1.953165,-2.055305,-2.166903,-2.316295,-2.371963,-2.429136,-2.428454,-2.458223,-2.456392,-2.472895
104996,2023-12-31 13:40:00,-0.784370,-0.882392,-1.052430,-1.092093,-1.238538,-1.432852,-1.518558,-1.584011,-1.736335,...,-2.046810,-2.138234,-2.240742,-2.376328,-2.432912,-2.480650,-2.488384,-2.546310,-2.531991,-2.535056
104997,2023-12-31 13:45:00,-1.249813,-1.324060,-1.460951,-1.483339,-1.603908,-1.784417,-1.870392,-1.909966,-2.039112,...,-2.515736,-2.644340,-2.756159,-2.878438,-2.955219,-3.025247,-3.048131,-3.102515,-3.068712,-3.075123
104998,2023-12-31 13:50:00,-1.112147,-1.153714,-1.279269,-1.253359,-1.310937,-1.483765,-1.548852,-1.566916,-1.681151,...,-2.470014,-2.527495,-2.610821,-2.684051,-2.738286,-2.825282,-2.807241,-2.826947,-2.802542,-2.884259


In [126]:
pred_df = pred_diff_df.copy()

In [127]:
pred_df.iloc[:,1:] = pred_diff_df.iloc[:,1:] + actual_dap_rolled.iloc[:,1:]

In [128]:
pred_df

,sced_time_stamp_local,1,2,3,4,5,6,7,8,9,...,111,112,113,114,115,116,117,118,119,120
0,2023-01-01 00:00:00,0.959186,0.934353,0.723857,0.808601,0.875203,1.037767,0.853202,0.750106,0.839050,...,6.283576,6.443935,6.432455,6.494379,6.417150,6.518711,6.786392,7.041070,7.056625,9.369008
1,2023-01-01 00:05:00,-2.543950,-2.561265,-2.647250,-2.462456,-2.333482,-2.138203,-2.238646,-2.228166,-2.026174,...,5.211310,5.437534,5.473638,5.577991,5.589973,5.716543,6.097568,6.283972,8.433748,8.535703
2,2023-01-01 00:10:00,-2.932860,-3.029690,-3.095066,-2.940293,-2.847942,-2.646128,-2.715639,-2.685369,-2.467252,...,5.077473,5.312374,5.399479,5.584123,5.643836,5.754697,6.018863,8.282166,8.159060,8.174580
3,2023-01-01 00:15:00,-2.842954,-2.952721,-3.008237,-2.866018,-2.821409,-2.605889,-2.623570,-2.574662,-2.902291,...,5.032102,5.278808,5.378662,5.600585,5.597565,5.612376,7.948886,7.972735,7.844919,7.813117
4,2023-01-01 00:20:00,-2.774014,-2.862975,-2.873166,-2.758672,-2.728463,-2.520902,-2.520643,-3.058440,-2.791864,...,4.981766,5.210392,5.305995,5.468157,5.347032,7.507195,7.559531,7.535611,7.419412,7.379155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104995,2023-12-31 13:35:00,11.222122,11.153139,11.003983,10.964763,10.406116,10.207102,10.122371,10.052753,9.886679,...,13.816735,13.714595,14.073497,13.924105,13.868437,13.811264,13.811946,13.782177,13.784008,13.767505
104996,2023-12-31 13:40:00,11.770530,11.672508,11.502470,11.055707,10.909263,10.714949,10.629243,10.563789,10.411465,...,13.723090,14.102166,13.999659,13.864072,13.807488,13.759750,13.752016,13.694090,13.708409,13.705344
104997,2023-12-31 13:45:00,11.305087,11.230840,10.686850,10.664461,10.543893,10.363383,10.277408,10.237834,10.108688,...,13.724664,13.596061,13.484241,13.361961,13.285181,13.215153,13.192268,13.137885,13.171688,13.165277
104998,2023-12-31 13:50:00,11.442753,10.994086,10.868531,10.894442,10.836864,10.664036,10.598948,10.580885,10.466649,...,13.770387,13.712905,13.629580,13.556349,13.502114,13.415118,13.433159,13.413453,13.437859,13.356141


In [129]:
pred_df.iloc[:,1:].values

array([[ 0.95918584,  0.9343531 ,  0.72385716, ...,  7.0410705 ,
         7.056625  ,  9.369008  ],
       [-2.5439498 , -2.5612648 , -2.64725   , ...,  6.2839723 ,
         8.433748  ,  8.535703  ],
       [-2.9328601 , -3.0296896 , -3.0950658 , ...,  8.282166  ,
         8.15906   ,  8.17458   ],
       ...,
       [11.305087  , 11.23084   , 10.68685   , ..., 13.137885  ,
        13.171688  , 13.165277  ],
       [11.442753  , 10.994086  , 10.868531  , ..., 13.413453  ,
        13.437859  , 13.356141  ],
       [10.150934  , 10.1608715 , 10.087503  , ..., 13.287161  ,
        13.345395  , 13.271947  ]], dtype=float32)

In [130]:
evaluate_prediction(pred_df.iloc[:,1:].values , actual_sl_rolled.iloc[:,1:].values, '')


MAE: 38.27
RMSE: 220.85




(38.27354045777775, 220.849050757651)

In [103]:
pred_df.to_csv('TimesFM_120_diff_sum_2023.csv')

In [14]:
pred_df = pd.read_csv('TimesFM_120_diff_sum_2023.csv')

# Storage Arbitrage
Perform storage arbitrage for WAKEWE_ALL node. Use the predicted system lambda to derive the best decisions to maximize profit.
The test year is 2023, so we need the nodal real-time price for that year.

In [15]:
# Storage parameters
bid_ahead = 1
horizon = int(look_ahead/12) - bid_ahead #in hours

Ts = 1/12  # hourly time step. If the price data is 5-min; Ts = 1/12. If it is hourly; Ts = 1
hr = int(1 / Ts)  # number of time steps in one hour

# Parameters
E = 1  # Energy rating in MWh
Pr = 1 / 2  # normalized power rating wrt energy rating
P = Pr * Ts  # actual power rating taking time step size into account
eta = 0.9  # efficiency
c = 10  # marginal discharge cost - degradation
initial_soc = 0.5

T = hr * horizon #the horizon as number of steps

In [ ]:
# optimization model
result_dfs = []

eS = np.zeros(T)  # initialize the SoC series
profit = np.zeros(T)  # initialize the profit series
revenue = np.zeros(T)  # initialize the revenue series

soc_var = cp.Variable(T, nonneg=True)
chr_var = cp.Variable(T, nonneg=True)
dis_var = cp.Variable(T, nonneg=True)
charge_decision = cp.Variable(T, boolean=True)
discharge_decision = cp.Variable(T, boolean=True)
discharge_active = cp.Variable(T, boolean=True)
M = 1e6  # A large constant
price_param = cp.Parameter(T)
soc_init_param = cp.Parameter()

cons = [
    soc_var[0] - soc_init_param == chr_var[0] * eta - dis_var[0] / eta,
    *[soc_var[i] - soc_var[i-1] == chr_var[i] * eta - dis_var[i] / eta for i in range(1, T)],
    *[chr_var[i] <= P for i in range(T)],
    *[soc_var[i] <= E for i in range(T)],
    *[charge_decision[i] + discharge_decision[i] <= 1 for i in range(T)],
    *[dis_var[i] <= discharge_active[i] * P for i in range(T)],
    *[dis_var[i] >= 0 for i in range(T)],
    *[price_param[i] >= -M * (1 - discharge_active[i]) for i in range(T)],
    *[price_param[i] <= M * discharge_active[i] for i in range(T)]
]

revenue_obj = cp.multiply(price_param, (dis_var - chr_var)) - c * dis_var
problem = cp.Problem(cp.Maximize(cp.sum(revenue_obj)), cons)


for start in tqdm(range(int(len(actual_rolled)-bid_ahead*12))):
    slice_actual = actual_rolled.iloc[start+bid_ahead*12, :T+1]
    slice_pred = pred_df.iloc[start, 1+bid_ahead*12:T+bid_ahead*12+1].copy()
    slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item

    price_param.value = slice_pred.values.astype(float)
    soc_init_param.value = initial_soc
    problem.solve(solver=cp.GUROBI)
    pS_dis = dis_var.value
    pS_chr = chr_var.value
    eS = soc_var.value
    revenue = slice_actual.iloc[1:].values.astype(float) * (pS_dis - pS_chr)
    profit = revenue - c * pS_dis
    storage_profile_sl = pd.Series({
        'time': slice_actual.iloc[0],
        'actual': slice_actual.iloc[1],
        'pred': slice_pred.iloc[0],
        'discharge': pS_dis[0],
        'charge': pS_chr[0],
        'soc': eS[0],
        'profit': profit[0],
        'revenue': revenue[0]})
    result_dfs.append(storage_profile_sl)
    initial_soc = eS[0]

final_result_lmp = pd.DataFrame(result_dfs)

final_result_lmp.to_csv(str(bid_ahead)+'hr_bidding_results'+str(T)+'_lmp_5min_TimesFM_diff.csv', index=False)



  0%|                                                                                                                                                              | 0/105000 [00:00<?, ?it/s]/tmp/ipykernel_3603405/2008716508.py:37: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 104988/105000 [41:14<00:00, 42.43it/s]


IndexError: single positional indexer is out-of-bounds

In [20]:
final_result_lmp = pd.DataFrame(result_dfs)

final_result_lmp.to_csv(str(bid_ahead)+'hr_bidding_results'+str(T)+'_lmp_5min_TimesFM_diff.csv', index=False)


In [23]:
final_result_lmp.profit.sum()

70039.59767752056

In [24]:
# Storage parameters
bid_ahead = 0
horizon = int(look_ahead/12) - bid_ahead #in hours

Ts = 1/12  # hourly time step. If the price data is 5-min; Ts = 1/12. If it is hourly; Ts = 1
hr = int(1 / Ts)  # number of time steps in one hour

# Parameters
E = 1  # Energy rating in MWh
Pr = 1 / 2  # normalized power rating wrt energy rating
P = Pr * Ts  # actual power rating taking time step size into account
eta = 0.9  # efficiency
c = 10  # marginal discharge cost - degradation
initial_soc = 0.5

T = hr * horizon #the horizon as number of steps

# optimization model
result_dfs = []

eS = np.zeros(T)  # initialize the SoC series
profit = np.zeros(T)  # initialize the profit series
revenue = np.zeros(T)  # initialize the revenue series

soc_var = cp.Variable(T, nonneg=True)
chr_var = cp.Variable(T, nonneg=True)
dis_var = cp.Variable(T, nonneg=True)
charge_decision = cp.Variable(T, boolean=True)
discharge_decision = cp.Variable(T, boolean=True)
discharge_active = cp.Variable(T, boolean=True)
M = 1e6  # A large constant
price_param = cp.Parameter(T)
soc_init_param = cp.Parameter()

cons = [
    soc_var[0] - soc_init_param == chr_var[0] * eta - dis_var[0] / eta,
    *[soc_var[i] - soc_var[i-1] == chr_var[i] * eta - dis_var[i] / eta for i in range(1, T)],
    *[chr_var[i] <= P for i in range(T)],
    *[soc_var[i] <= E for i in range(T)],
    *[charge_decision[i] + discharge_decision[i] <= 1 for i in range(T)],
    *[dis_var[i] <= discharge_active[i] * P for i in range(T)],
    *[dis_var[i] >= 0 for i in range(T)],
    *[price_param[i] >= -M * (1 - discharge_active[i]) for i in range(T)],
    *[price_param[i] <= M * discharge_active[i] for i in range(T)]
]

revenue_obj = cp.multiply(price_param, (dis_var - chr_var)) - c * dis_var
problem = cp.Problem(cp.Maximize(cp.sum(revenue_obj)), cons)


for start in tqdm(range(int(len(actual_rolled)-bid_ahead*12))):
    slice_actual = actual_rolled.iloc[start+bid_ahead*12, :T+1]
    slice_pred = pred_df.iloc[start, 1+bid_ahead*12:T+bid_ahead*12+1].copy()
    slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item

    price_param.value = slice_pred.values.astype(float)
    soc_init_param.value = initial_soc
    problem.solve(solver=cp.GUROBI)
    pS_dis = dis_var.value
    pS_chr = chr_var.value
    eS = soc_var.value
    revenue = slice_actual.iloc[1:].values.astype(float) * (pS_dis - pS_chr)
    profit = revenue - c * pS_dis
    storage_profile_sl = pd.Series({
        'time': slice_actual.iloc[0],
        'actual': slice_actual.iloc[1],
        'pred': slice_pred.iloc[0],
        'discharge': pS_dis[0],
        'charge': pS_chr[0],
        'soc': eS[0],
        'profit': profit[0],
        'revenue': revenue[0]})
    result_dfs.append(storage_profile_sl)
    initial_soc = eS[0]

final_result_lmp = pd.DataFrame(result_dfs)

final_result_lmp.to_csv(str(bid_ahead)+'hr_bidding_results'+str(T)+'_lmp_5min_TimesFM_diff.csv', index=False)



  0%|                                                                                                                                                              | 0/105000 [00:00<?, ?it/s]/tmp/ipykernel_3603405/2520134126.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 105000/105000 [44:17<00:00, 39.51it/s]


In [25]:
final_result_lmp.profit.sum()

70545.92182937241

In [26]:
# Storage parameters
bid_ahead = 2
horizon = int(look_ahead/12) - bid_ahead #in hours

Ts = 1/12  # hourly time step. If the price data is 5-min; Ts = 1/12. If it is hourly; Ts = 1
hr = int(1 / Ts)  # number of time steps in one hour

# Parameters
E = 1  # Energy rating in MWh
Pr = 1 / 2  # normalized power rating wrt energy rating
P = Pr * Ts  # actual power rating taking time step size into account
eta = 0.9  # efficiency
c = 10  # marginal discharge cost - degradation
initial_soc = 0.5

T = hr * horizon #the horizon as number of steps

# optimization model
result_dfs = []

eS = np.zeros(T)  # initialize the SoC series
profit = np.zeros(T)  # initialize the profit series
revenue = np.zeros(T)  # initialize the revenue series

soc_var = cp.Variable(T, nonneg=True)
chr_var = cp.Variable(T, nonneg=True)
dis_var = cp.Variable(T, nonneg=True)
charge_decision = cp.Variable(T, boolean=True)
discharge_decision = cp.Variable(T, boolean=True)
discharge_active = cp.Variable(T, boolean=True)
M = 1e6  # A large constant
price_param = cp.Parameter(T)
soc_init_param = cp.Parameter()

cons = [
    soc_var[0] - soc_init_param == chr_var[0] * eta - dis_var[0] / eta,
    *[soc_var[i] - soc_var[i-1] == chr_var[i] * eta - dis_var[i] / eta for i in range(1, T)],
    *[chr_var[i] <= P for i in range(T)],
    *[soc_var[i] <= E for i in range(T)],
    *[charge_decision[i] + discharge_decision[i] <= 1 for i in range(T)],
    *[dis_var[i] <= discharge_active[i] * P for i in range(T)],
    *[dis_var[i] >= 0 for i in range(T)],
    *[price_param[i] >= -M * (1 - discharge_active[i]) for i in range(T)],
    *[price_param[i] <= M * discharge_active[i] for i in range(T)]
]

revenue_obj = cp.multiply(price_param, (dis_var - chr_var)) - c * dis_var
problem = cp.Problem(cp.Maximize(cp.sum(revenue_obj)), cons)


for start in tqdm(range(int(len(actual_rolled)-bid_ahead*12))):
    slice_actual = actual_rolled.iloc[start+bid_ahead*12, :T+1]
    slice_pred = pred_df.iloc[start, 1+bid_ahead*12:T+bid_ahead*12+1].copy()
    slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item

    price_param.value = slice_pred.values.astype(float)
    soc_init_param.value = initial_soc
    problem.solve(solver=cp.GUROBI)
    pS_dis = dis_var.value
    pS_chr = chr_var.value
    eS = soc_var.value
    revenue = slice_actual.iloc[1:].values.astype(float) * (pS_dis - pS_chr)
    profit = revenue - c * pS_dis
    storage_profile_sl = pd.Series({
        'time': slice_actual.iloc[0],
        'actual': slice_actual.iloc[1],
        'pred': slice_pred.iloc[0],
        'discharge': pS_dis[0],
        'charge': pS_chr[0],
        'soc': eS[0],
        'profit': profit[0],
        'revenue': revenue[0]})
    result_dfs.append(storage_profile_sl)
    initial_soc = eS[0]

final_result_lmp = pd.DataFrame(result_dfs)

final_result_lmp.to_csv(str(bid_ahead)+'hr_bidding_results'+str(T)+'_lmp_5min_TimesFM_diff.csv', index=False)



  0%|                                                                                                                                                                                   | 0/104976 [00:00<?, ?it/s]/tmp/ipykernel_3603405/1099204449.py:54: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  slice_pred.iloc[0] = slice_actual[1]  # Replace the first item with actual_rolled's first item
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 104976/104976 [39:13<00:00, 44.59it/s]


In [27]:
final_result_lmp.profit.sum()

68097.01979012345